## 실험 내용
- 이미 불가능하다고 판단된 주문의 조합은 다시 확인하지 않을 때 비용이 최소 같아야 하는데 더 높아지는 문제 해결하기

### 기존 코드 결과 확인

In [19]:
%load_ext autoreload
%autoreload 2

from util_0803_1 import *

problem_file = r'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_6.json'

## ------------------- 기본 변수 할당 ---------------------------

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

inf = float('inf')

timelimit = 60

## ------------------- 초기 상태 할당 코드 -------------------------

car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

init_availables = [rider.available_number for rider in ALL_RIDERS]

all_bundles = []
for ord in ALL_ORDERS:
    new_bundle = Bundle(ALL_ORDERS, car_rider, [ord.id], [ord.id], ord.volume, DIST[ord.id, ord.id+K])
    car_rider.available_number -= 1
    all_bundles.append(new_bundle)

## ------------------  주문 조합 가능성 행렬 제작하기 -----------------------------

order_comb_possibility = [[True] * K for _ in range(K)]

for i in range(len(all_bundles)):
    for j in range(i + 1, len(all_bundles)):
        bundle1 = all_bundles[i]
        bundle2 = all_bundles[j]

        order_num1 = bundle1.shop_seq[0]
        order_num2 = bundle2.shop_seq[0]

        ip = try_merging_bundles_by_dist(K, DIST, ALL_ORDERS, ALL_RIDERS, bundle1, bundle2)

        if not ip:
            order_comb_possibility[order_num1][order_num2] = False
            order_comb_possibility[order_num2][order_num1] = False

# optimized_order_perms = [dict(), dict(), dict()] # optimized_order_perms[rider_i] = {orders_sorted: 최적 번들}

min_init_cost = inf
min_init_cost_bundles = []
min_init_cost_rider_availables = []

weight_grid = [(1, -1), (1, -1.5), (1, -2), (1, -2.5), (1, -3), (1, -3.5), (0.5, 1), (0.5, 2), (0.5, 3)]

temp_process_time = -1
for weight1, weight2 in weight_grid:

    bundles, result_rider_availables, cost = get_init_bundle_4_order_bundle_prefered_with_reassigning_riders(
            K, ALL_RIDERS, ALL_ORDERS, DIST, init_availables, weight1, weight2, order_comb_possibility)

    if cost < min_init_cost:
        min_init_cost = cost
        min_init_cost_bundles = bundles
        min_init_cost_rider_availables = result_rider_availables

    print(weight1, weight2, cost)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
1 -1 2660.1369999999993
1 -1.5 2678.6279999999992
1 -2 2713.6219999999994
1 -2.5 2687.855333333333
1 -3 2792.2129999999993
1 -3.5 2772.0459999999994
0.5 1 2592.1979999999994
0.5 2 2806.5233333333335
0.5 3 2877.197


### 주문 조합 한 번만 보는 방식의 코드 결과 확인

In [5]:
%load_ext autoreload
%autoreload 2

from util_0803_3 import *

problem_file = r'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_6.json'

## ------------------- 기본 변수 할당 ---------------------------

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

inf = float('inf')

timelimit = 60

## ------------------- 초기 상태 할당 코드 -------------------------

car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

init_availables = [rider.available_number for rider in ALL_RIDERS]

all_bundles = []
for ord in ALL_ORDERS:
    new_bundle = Bundle(ALL_ORDERS, car_rider, [ord.id], [ord.id], ord.volume, DIST[ord.id, ord.id+K])
    car_rider.available_number -= 1
    all_bundles.append(new_bundle)

## ------------------  주문 조합 가능성 행렬 제작하기 -----------------------------

order_comb_possibility = [[True] * K for _ in range(K)]

for i in range(len(all_bundles)):
    for j in range(i + 1, len(all_bundles)):
        bundle1 = all_bundles[i]
        bundle2 = all_bundles[j]

        order_num1 = bundle1.shop_seq[0]
        order_num2 = bundle2.shop_seq[0]

        ip = try_merging_bundles_by_dist(K, DIST, ALL_ORDERS, ALL_RIDERS, bundle1, bundle2)

        if not ip:
            order_comb_possibility[order_num1][order_num2] = False
            order_comb_possibility[order_num2][order_num1] = False

optimized_order_perms = [dict(), dict(), dict()] # optimized_order_perms[rider_i] = {orders_sorted: 최적 번들}

min_init_cost = inf
min_init_cost_bundles = []
min_init_cost_rider_availables = []

weight_grid = [(1, -1), (1, -1.5), (1, -2), (1, -2.5), (1, -3), (1, -3.5), (0.5, 1), (0.5, 2), (0.5, 3)]

temp_process_time = -1
for weight1, weight2 in weight_grid:

    bundles, result_rider_availables, cost = get_init_bundle_4_order_bundle_prefered_with_reassigning_riders(
            K, ALL_RIDERS, ALL_ORDERS, DIST, init_availables, weight1, weight2, order_comb_possibility, optimized_order_perms)

    if cost < min_init_cost:
        min_init_cost = cost
        min_init_cost_bundles = bundles
        min_init_cost_rider_availables = result_rider_availables

    print(weight1, weight2, cost)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
1 -1 2659.666999999999
1 -1.5 2712.7626666666656
1 -2 2715.376666666666
1 -2.5 2741.1639999999993
1 -3 2794.761
1 -3.5 2809.4839999999995
0.5 1 2667.0
0.5 2 2793.266
0.5 3 2863.2056666666667


### 디버깅

In [20]:
%load_ext autoreload
%autoreload 2

from util_0803_4 import *

problem_file = r'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_6.json'

## ------------------- 기본 변수 할당 ---------------------------

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

inf = float('inf')

timelimit = 60

## ------------------- 초기 상태 할당 코드 -------------------------

car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

init_availables = [rider.available_number for rider in ALL_RIDERS]

all_bundles = []
for ord in ALL_ORDERS:
    new_bundle = Bundle(ALL_ORDERS, car_rider, [ord.id], [ord.id], ord.volume, DIST[ord.id, ord.id+K])
    car_rider.available_number -= 1
    all_bundles.append(new_bundle)

## ------------------  주문 조합 가능성 행렬 제작하기 -----------------------------

order_comb_possibility = [[True] * K for _ in range(K)]

for i in range(len(all_bundles)):
    for j in range(i + 1, len(all_bundles)):
        bundle1 = all_bundles[i]
        bundle2 = all_bundles[j]

        order_num1 = bundle1.shop_seq[0]
        order_num2 = bundle2.shop_seq[0]

        ip = try_merging_bundles_by_dist(K, DIST, ALL_ORDERS, ALL_RIDERS, bundle1, bundle2)

        if not ip:
            order_comb_possibility[order_num1][order_num2] = False
            order_comb_possibility[order_num2][order_num1] = False

optimized_order_perms = [dict(), dict(), dict()] # optimized_order_perms[rider_i] = {orders_sorted: 최적 번들}

min_init_cost = inf
min_init_cost_bundles = []
min_init_cost_rider_availables = []

weight_grid = [(1, -1), (1, -1.5), (1, -2), (1, -2.5), (1, -3), (1, -3.5), (0.5, 1), (0.5, 2), (0.5, 3)]

temp_process_time = -1
for weight1, weight2 in weight_grid:

    bundles, result_rider_availables, cost = get_init_bundle_4_order_bundle_prefered_with_reassigning_riders(
            K, ALL_RIDERS, ALL_ORDERS, DIST, init_availables, weight1, weight2, order_comb_possibility, optimized_order_perms)

    if cost < min_init_cost:
        min_init_cost = cost
        min_init_cost_bundles = bundles
        min_init_cost_rider_availables = result_rider_availables

    print(weight1, weight2, cost)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
1 -1 2660.1369999999993
1 -1.5 2678.6279999999992
1 -2 2713.6219999999994
1 -2.5 2687.855333333333
1 -3 2792.2129999999993
1 -3.5 2772.0459999999994
0.5 1 2592.1979999999994
0.5 2 2806.5233333333335
0.5 3 2877.197


In [ ]:
# 기존 코드
1 -1 2660.1369999999993
1 -1.5 2678.6279999999992
1 -2 2713.6219999999994
1 -2.5 2687.855333333333
1 -3 2792.2129999999993
1 -3.5 2772.0459999999994
0.5 1 2592.1979999999994
0.5 2 2806.5233333333335
0.5 3 2877.197

# 디버깅 전 수정 코드
1 -1 2659.666999999999
1 -1.5 2712.7626666666656
1 -2 2715.376666666666
1 -2.5 2741.1639999999993
1 -3 2794.761
1 -3.5 2809.4839999999995
0.5 1 2667.0
0.5 2 2793.266
0.5 3 2863.2056666666667

# 디버깅 후 수정 코드
1 -1 2659.666999999999
1 -1.5 2712.7626666666656
1 -2 2715.376666666666
1 -2.5 2741.1973333333326
1 -3 2794.761
1 -3.5 2764.549666666666
0.5 1 2667.0
0.5 2 2793.266
0.5 3 2863.2056666666667